In [13]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [ ]:
%sendtofile --source SI7021_funcs.py
%sendtofile --source device_detection.py

In [2]:
from OLED_driver import i2c, fbuff, oledshow

In [14]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow
import time
time.sleep(0.5)
from device_detection import IdentifyI2CDevice

devices = IdentifyI2CDevice(i2c)
dev = devices[0] if devices else "nothing"
fbuff.fill(0)
fbuff.text(dev, 0, 24, 1)
oledshow()

if dev == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid: %.1f"%h, 8, 20, 1)
        fbuff.text("temp:  %.1f"%t, 8, 30, 1)
        oledshow()

if dev == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist: %.1f"%d, 8, 20, 1)
        oledshow()


Sent 31 lines (839 bytes) to main.py.
